## Row reduction and Lu Decomposition

In [2]:
import numpy as np

Solve a system of equations with a matrix system with Python.

The order is important, because in matrix multiplication, order matters.


$$A=\begin{vmatrix}1 & 1\\
2 & 4
\end{vmatrix}$$


$$X=\begin{vmatrix}a\\b
\end{vmatrix}$$


$$B=\begin{vmatrix}35\\
94
\end{vmatrix}$$

AX = B \
A<sup>-1</sup>AX = A<sup>-1</sup>B \
X = A<sup>-1</sup>B

to solve for x, pre-multiply the inverse of A by B

In [4]:
A = np.array([[1,1],[2,4]])
B = np.array([[35], [94]])

# solve for x
X = np.linalg.inv(A)@B
print(X)

[[23.]
 [12.]]


### Row reduction

Row reduction means iteratively apply two operations-scalar multiplication and addition, to rows of a matrix. Row reduction relies on the same principle as adding equations to other equations within a system.

<i>The goal of row reduction is to transform a dense matrix into an upper-triangular matrix.</i> \
The upper-triangular matrix that results from row reduction is called the <i>echelon form</i> of the matrix

As the matrix before and after the row reduction is linked by a linear transformation, it is possible to express row reduction with a matrix  \
As there are many (or infinite) row echelons form for the same matrix, two of them that are unique, reduced row echelon form and U are the most important ones.

There is no direct function in Pyhon to calculate a row echelon form of a matrix

### Gaussian elimination

It is possible to solve a matrix system of equation for X without using iversion. This method is called Gaussian Elimination.

Gaussian elimination consists in:
- augment the matrix of coefficients by the vector of constants
- row reduce to echelon form
- use back substitution to solve for each variable in turn

